In [1]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3

In [17]:
DB_PATH = "../anaslo_02/anaslo_02.db"
DB_PATH = r"C:\python\dataOnline\anaslo_02\anaslo_02.db"
SPREADSHEET_ID = "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8"

In [18]:
# スプレッドシート認証設定
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
jsonf = r"C:\python\dataOnline\anaslo_02\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

In [27]:
# # Table name 取得
# conn = sqlite3.connect(DB_PATH)
# cursor = conn.cursor()
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
# tables = cursor.fetchall()
# conn.close()

# print(tables)

In [30]:
# 検索キーワードよりホール名取得
SEARCH_WORD = "EXA"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT hall_id, name FROM halls WHERE name LIKE ?", ('%' + SEARCH_WORD + '%',))
results = cursor.fetchall()

# 結果表示
if results:
    print(f"🔍 '{SEARCH_WORD}' を含むホール名の検索結果:")
    for hall_id, hall_name in results:
        print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")
    
query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
  AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
df['date'] = pd.to_datetime(df['date'])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["day"] = df["date"].dt.day
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.weekday

conn.close()

df.shape

🔍 'EXA' を含むホール名の検索結果:
 - hall_name: EXA FIRST, hall_id: 108


(51376, 14)

In [ ]:
# モデル別日ごと
sheet_name = "DAILY_MODELS"
df_tmp = df.copy()
# メダル
medals = df_tmp.pivot_table(index=["model_name"], columns="day", values="medals", aggfunc="sum", margins=True, margins_name="Total")
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
# RB＿RATE
game = df_tmp.pivot_table(index=["model_name"], columns="day", values="game", aggfunc="sum", margins=True, margins_name="Total")
rb = df_tmp.pivot_table(index=["model_name"], columns="day", values="RB", aggfunc="sum", margins=True, margins_name="Total")
rb_rate = (game / rb).round(1)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna("")
# WRITE
sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)

In [ ]:
# マシン別メダル
sheet_name = "DAILY_UNIT"
MACHINE_NAME = "マイジャグラーV"
df_tmp = df[df["model_name"] == MACHINE_NAME]
# メダル数
medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="day", values="medals", aggfunc="sum", margins=True, margins_name="Total")
medals["rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
medals.shape
# RB_rate 作成
game = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="day", values="game", aggfunc="sum", margins=True, margins_name="Total")
rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="day", values="RB", aggfunc="sum", margins=True, margins_name="Total")
rb_rate = (game / rb).round(1)
rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna(0)
rb_rate.shape

spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)


(82, 33)

In [ ]:
# 日付指定
DAY = 12
SPACE_ROW = 2

next_row = 1
sheet_name = f"DAY{DAY}"
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()

for model in df["model_name"].unique()[:-1]:
    
    df_tmp = df[(df["day"] == DAY) & (df["model_name"] == model)].copy()
    df_tmp["date"] = df_tmp["date"].dt.strftime('%Y-%m-%d')
    
    # メダル数
    medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum", margins=True, margins_name="Total")
    medals["rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    print(f"{model}_{medals.shape}")

    # RB_RATE
    game = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="game", aggfunc="sum", margins=True, margins_name="Total")
    rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="RB", aggfunc="sum", margins=True, margins_name="Total")
    rb_rate = (game / rb).round(1)
    rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
    rb_rate = rb_rate.fillna(0)
    print(f"{model}_{rb_rate.shape}")
    
    # スプレッドシート追記
    sheet_name = f"DAY{DAY}"
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += (medals.shape[0] + SPACE_ROW+3)  # 追記行
    set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += (medals.shape[0] + SPACE_ROW+3)  # 追記行

(82, 13)
(82, 13)
(15, 13)
(15, 13)
(8, 10)
(8, 10)
(50, 13)
(50, 13)
(9, 6)
(9, 6)
(33, 13)
(33, 13)
(40, 13)
(40, 13)
(7, 3)
(7, 3)
